Hello!

The following code:

1) Counts the number of words that occur only once in a number of Renaissance poems and compares that number to other  literary texts.

2) Plots the rank-frequency distribution of word tokens for each work.

In [ ]:
# These are the libraries I used 

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import os
from collections import Counter
%matplotlib inline
import nltk
from nltk.corpus import stopwords
import spacy
import numpy as np
import os
import pyLDAvis.sklearn
import re
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.decomposition import LatentDirichletAllocation, PCA
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle

import plotly.offline as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly
plotly.offline.init_notebook_mode(connected=True)
import scipy
import collections

from cltk.corpus.utils.importer import CorpusImporter

In [ ]:
# This downloads the CLTK pre-trained models for Latin and the LL (Latin Library) corpus

my_latin_downloader = CorpusImporter('latin')
my_latin_downloader.import_corpus('latin_text_latin_library')
my_latin_downloader.import_corpus('latin_models_cltk')

In [ ]:
# This takes four Latin poems from directories in the downloaded Latin Library corpus and opens them 

from cltk.corpus.readers import get_corpus_reader
latinlibrary = get_corpus_reader(corpus_name = 'latin_text_latin_library', language = 'latin')
files = latinlibrary.fileids()
ll = latinlibrary.raw(files)

## Loop through the files in the author folders and call the .raw method on the 3 lists

# De Rerum Natura 
dererumnatura_files = [file for file in files if 'lucretius/lucretius' in file]
dererumnatura = latinlibrary.raw(dererumnatura_files)
# Vergil's Aeneid
aeneid_files = [file for file in files if 'vergil/aen' in file]
aeneid = latinlibrary.raw(aeneid_files)
# Statius' Thebaid 
thebaid_files = [file for file in files if 'statius/theb' in file]
thebaid = latinlibrary.raw(thebaid_files)
# Lucan's Pharsalia
pharsalia_files = [file for file in files if 'lucan/lucan' in file]
pharsalia = latinlibrary.raw(pharsalia_files)

In [ ]:
# This prepares the CLTK Latin lemmatizer to use on the various Latin texts

from cltk.tokenize.sentence import TokenizeSentence
corpus_importer = CorpusImporter('latin')
tokenizer = TokenizeSentence('latin')

In [ ]:
os.chdir("/Users/matthewmason/cltk/corpus/")

In [ ]:
# This assigns the MP texts to variables. The .csv included data from 10 MP texts

baldus = open("Baldus1552.txt", encoding = "utf-8").read()

In [ ]:
os.chdir("/Users/matthewmason/cltk/corpus2/neo-Latin")

In [ ]:
# The neo-Latin poems

africa = open("africa.txt", encoding = "utf-8").read()
christiad = open("christiad.txt", encoding = "utf-8").read()

In [ ]:
os.chdir("/Users/matthewmason/cltk/corpus2/italian")

In [ ]:
# The Italian poems

commedia = open("commedia.txt", encoding = "utf-8").read()
morgante = open("morgante.txt", encoding = "utf-8").read()
orlando_innamorato = open("orlando_innamorato.txt", encoding = "utf-8").read()
orlando_furioso = open("orlando_furioso.txt", encoding = "utf-8").read()
italia_liberata = open("italia_liberata.txt", encoding = "utf-8").read()
gerusalemme_liberata = open("gerusalemme_liberata.txt", encoding = "utf-8").read()

In [ ]:
os.chdir("/Users/matthewmason/cltk/corpus2/english")

In [ ]:
# The English poems, too

faerie_queen = open("faerie_queen.txt", encoding = "utf-8").read()
venus_and_adonis = open("venus_and_adonis.txt", encoding = "utf-8").read()
paradise_lost = open("paradise_lost.txt", encoding = "utf-8").read()
finnagans_wake = open("finnagans_wake.txt", encoding = "utf-8").read()
ulysses = open("ulysses.txt", encoding = "utf-8").read()

In [ ]:
from cltk.tokenize.word import WordTokenizer
from cltk.stop.latin import STOPS_LIST
from cltk.lemmatize.latin.backoff import BackoffLatinLemmatizer
import nltk
from nltk.corpus import stopwords

word_tokenizer = WordTokenizer('latin')
lemmatizer = BackoffLatinLemmatizer()

In [ ]:
# This lowers the capital letters  

baldus_lowered = baldus.lower()

dererumnatura_lowered = dererumnatura.lower()
aeneid_lowered = aeneid.lower()
thebaid_lowered = thebaid.lower() 
pharsalia_lowered = pharsalia.lower()

africa_lowered = africa.lower()
christiad_lowered = christiad.lower() 

commedia_lowered = commedia.lower()
morgante_lowered = morgante.lower()
orlando_innamorato_lowered = orlando_innamorato.lower()
orlando_furioso_lowered = orlando_furioso.lower()
italia_liberata_lowered = italia_liberata.lower()
gerusalemme_liberata_lowered = gerusalemme_liberata.lower()

faerie_queen_lowered = faerie_queen.lower()
venus_and_adonis_lowered = venus_and_adonis.lower()
paradise_lost_lowered = paradise_lost.lower()
finnagans_wake_lowered = finnagans_wake.lower()
ulysses_lowered = ulysses.lower()

# This tokenizes, removes punctuation, stop words. For the .csv I turned the lists of tokens into strings.

baldus_tokens = word_tokenizer.tokenize(baldus_lowered)
baldus_tokens_nopun = [token for token in baldus_tokens if token not in ['.', ',', ':', ';', '—', '?', '!','"', "*", "„", ")", "(", "-", "»"]]
baldus_no_stops = [word for word in baldus_tokens_nopun if not word in STOPS_LIST]
dererumnatura_tokens = word_tokenizer.tokenize(dererumnatura_lowered)
dererumnatura_tokens_nopun = [token for token in dererumnatura_tokens if token not in ['.', ',', ':', ';', '—', '?', '!','"', "*", "„", ")", "(", "-", "»"]]
dererumnatura_no_stops = [word for word in dererumnatura_tokens_nopun if not word in STOPS_LIST]
dererumnatura_lemmas = lemmatizer.lemmatize(dererumnatura_no_stops)
aeneid_tokens = word_tokenizer.tokenize(aeneid_lowered)
aeneid_tokens_nopun = [token for token in aeneid_tokens if token not in ['.', ',', ':', ';', '—', '?', '!','"', "*", "„", ")", "(", "-", "»"]] 
aeneid_no_stops = [word for word in aeneid_tokens_nopun if not word in STOPS_LIST]
thebaid_tokens = word_tokenizer.tokenize(thebaid_lowered)
thebaid_tokens_nopun = [token for token in thebaid_tokens if token not in ['.', ',', ':', ';', '—', '?', '!','"', "*", "„", ")", "(", "-", "»"]]
thebaid_no_stops = [word for word in thebaid_tokens_nopun if not word in STOPS_LIST]
pharsalia_tokens = word_tokenizer.tokenize(pharsalia_lowered)
pharsalia_tokens_nopun = [token for token in pharsalia_tokens if token not in ['.', ',', ':', ';', '—', '?', '!','"', "*", "„", ")", "(", "-", "»"]]
pharsalia_no_stops = [word for word in pharsalia_tokens_nopun if not word in STOPS_LIST]
africa_tokens = word_tokenizer.tokenize(africa_lowered)
africa_tokens_nopun = [token for token in africa_tokens if token not in ['.', ',', ':', ';', '—', '?', '!','"', "*", "„", ")", "(", "-", "»"]]
africa_no_stops = [word for word in africa_tokens_nopun if not word in STOPS_LIST]

In [ ]:
# This lemmatizes the Italian texts with SpaCy and it_core_news_sm

nlp = spacy.load("it_core_news_sm", disable=["ner", "textcat"])
nlp.max_length = 1500000

commedia_tokens = lemmatize(commedia_lowered)
commedia_tokens_nopun = [token for token in commedia_tokens if token not in ["'", '\n\n \n\n', '\n\n', '\n', '.', ',', ':', ';', '—', '?', '!','"', "*", "„", ")", "(", "-", "»", "«"]]
commedia_no_stops = [word for word in commedia_tokens_nopun if word not in stopwords.words('italian')]
commedia_string_no_stops = " ".join(commedia_no_stops)
morgante_tokens = lemmatize(morgante_lowered)
morgante_tokens_nopun = [token for token in morgante_tokens if token not in ["'", '\n\n \n\n', '\n\n', '\n', '.', ',', ':', ';', '—', '?', '!','"', "*", "„", ")", "(", "-", "»", "«"]]
morgante_no_stops = [word for word in morgante_tokens_nopun if word not in stopwords.words('italian')]
morgante_string_no_stops = " ".join(morgante_no_stops)
orlando_innamorato_tokens = lemmatize(orlando_innamorato_lowered) 
orlando_innamorato_tokens_nopun = [token for token in orlando_innamorato_tokens if token not in ["'", '\n\n \n\n', '\n\n', '\n', '.', ',', ':', ';', '—', '?', '!','"', "*", "„", ")", "(", "-", "»", "«"]]
orlando_innamorato_no_stops = [word for word in orlando_innamorato_tokens_nopun if word not in stopwords.words('italian')]
orlando_innamorato_string_no_stops = " ".join(orlando_innamorato_no_stops)
orlando_furioso_tokens = lemmatize(orlando_furioso_lowered) 
orlando_furioso_tokens_nopun = [token for token in orlando_furioso_tokens if token not in ["'", '\n\n \n\n', '\n\n\n', '\n\n', '\n', '.', ',', ':', ';', '—', '?', '!','"', "*", "„", ")", "(", "-", "»", "«"]]
orlando_furioso_no_stops = [word for word in orlando_furioso_tokens_nopun if word not in stopwords.words('italian')]
orlando_furioso_string_no_stops = " ".join(orlando_furioso_no_stops)
italia_liberata_tokens = lemmatize(italia_liberata_lowered)
italia_liberata_tokens_nopun = [token for token in italia_liberata_tokens if token not in ["'", '\n\n \n\n', '\n\n', '\n', '.', ',', ':', ';', '—', '?', '!','"', "*", "„", ")", "(", "-", "»", "«"]]
italia_liberata_no_stops = [word for word in italia_liberata_tokens_nopun if word not in stopwords.words('italian')]
italia_liberata_string_no_stops = " ".join(italia_liberata_no_stops)
gerusalemme_liberata_tokens = lemmatize(gerusalemme_liberata_lowered)
gerusalemme_liberata_tokens_nopun = [token for token in gerusalemme_liberata_tokens if token not in ["'", '\n\n \n\n', '\n\n', '\n', '.', ',', ':', ';', '—', '?', '!','"', "*", "„", ")", "(", "-", "»", "«"]]
gerusalemme_liberata_no_stops = [word for word in gerusalemme_liberata_tokens_nopun if word not in stopwords.words('italian')]
gerusalemme_liberata_string_no_stops = " ".join(gerusalemme_liberata_no_stops)

In [ ]:
# This load the model en_core_web_sm and lemmatizes the English poems

nlp = spacy.load("en_core_web_sm", disable=["ner", "textcat"])
nlp.max_length = 1600000

faerie_queen_tokens = lemmatize(faerie_queen_lowered)
faerie_queen_nopun = [token for token in faerie_queen_tokens if token not in ["'", '\n\n \n\n', '\n\n', '\n', '.', ',', ':', ';', '—', '?', '!','"', "*", "„", ")", "(", "-", "»", "«"]]
faerie_queen_no_stops = [word for word in faerie_queen_nopun if word not in stopwords.words('english')]
faerie_queen_string_no_stops = " ".join(faerie_queen_no_stops)
venus_and_adonis_tokens = lemmatize(venus_and_adonis_lowered)
venus_and_adonis_nopun = [token for token in venus_and_adonis_tokens if token not in ["'", '\n\n \n\n', '\n\n', '\n', '.', ',', ':', ';', '—', '?', '!','"', "*", "„", ")", "(", "-", "»", "«"]]
venus_and_adonis_no_stops = [word for word in venus_and_adonis_nopun if word not in stopwords.words('english')]
venus_and_adonis_string_no_stops = " ".join(venus_and_adonis_no_stops)
finnagans_wake_tokens = lemmatize(finnagans_wake_lowered)
finnagans_wake_nopun = [token for token in finnagans_wake_tokens if token not in ["'", '\n\n \n\n', '\n\n', '\n', '.', ',', ':', ';', '—', '?', '!','"', "*", "„", ")", "(", "-", "»", "«"]]
finnagans_wake_no_stops = [word for word in finnagans_wake_nopun if word not in stopwords.words('english')]
finnagans_wake_string_no_stops = " ".join(finnagans_wake_no_stops)
ulysses_tokens = lemmatize(ulysses_lowered)
ulysses_nopun = [token for token in ulysses_tokens if token not in ["'", '\n\n \n\n', '\n\n', '\n', '.', ',', ':', ';', '—', '?', '!','"', "*", "„", ")", "(", "-", "»", "«"]]
ulysses_no_stops = [word for word in ulysses_nopun if word not in stopwords.words('english')]
ulysses_string_no_stops = " ".join(ulysses_no_stops)
paradise_lost_tokens = lemmatize(paradise_lost_lowered)
paradise_lost_nopun = [token for token in paradise_lost_tokens if token not in ["'", '\n\n \n\n', '\n\n', '\n', '.', ',', ':', ';', '—', '?', '!','"', "*", "„", ")", "(", "-", "»", "«"]]
paradise_lost_no_stops = [word for word in paradise_lost_nopun if word not in stopwords.words('english')]
paradise_lost_string_no_stops = " ".join(paradise_lost_no_stops)

In [ ]:
# make a .csv

title = ["De Rerum Natura", "Aeneid", "Pharsalia", "Thebaid", "Africa", "Christiad", "De Partu Virginis", "Commedia", "Morgante," "Orlando innamorato", "Orlando furioso", "Italia liberata" "Gerusalemme liberata", "Macaronea", "Ad Magnificus Dominus Gasparus Vescontus", "Baldus", "Eglogae", "Macaronea Contra Macaroneam Bassani", "Macaronea Contra Savoynos", "Moschaea", "Nobile Vigonze Opus", "Vergiliana", "Zanitonella"]
date = ["99-55 BC", "29-19 BC", "61-65", "45-96", "1338-1374", "1535", "1526", "1321", "1478", "1483-1495", "1516-1532", "1547", "1581", "?", "?", "1552", "1517", "?", "?", "1521", "1502", "?", "1521"]
token_num = [len(set(dererumnatura_no_stops)), len(set(aeneid_no_stops)), len(set(pharsalia_no_stops)), len(set(thebaid_no_stops)), len(set(christiad_no_stops)), len(set(de_partu_virginis_no_stops)), len(set(commedia_no_stops)), len(set(morgante_no_stops)), len(set(orlando_innamorato_no_stops)), len(set(orlando_furioso_no_stops)), len(set(italia_liberata_no_stops)), len(set(gerusalemme_liberata_no_stops)), len(set(macaronea_no_stops)), len(set(magnificus_no_stops)), len(set(baldus_no_stops)), len(set(eglogae_no_stops)), len(set(contra_no_stops)), len(set(contra2_no_stops)), len(set(moschaea_no_stops)), len(set(vigonze_no_stops)), len(set(vergiliana_no_stops)), len(set(zanitonella_no_stops))]
fulltext = [dererumnatura, aeneid, pharsalia, thebaid, africa, christiad, de_partu_virginis, commedia, morgante, orlando_innamorato, orlando_furioso, italia_liberata, gerusalemme_liberata, macaronea, magnificus, baldus, eglogae, contra, contra2, moschaea, vigonze, vergiliana, zanitonella]
token_strings = [dererumnatura_string_no_stops, pharsalia_string_no_stops, thebaid_string_no_stops, africa_string_no_stops, christiad_string_no_stops, de_partu_virginis_string_no_stops, commedia_string_no_stops, morgante_string_no_stops, orlando_innamorato_string_no_stops, orlando_furioso_string_no_stops, italia_liberata_string_no_stops, gerusalemme_liberata_string_no_stops, macaronea_string_no_stops, magnificus_string_no_stops, baldus_string_no_stops, eglogae_string_no_stops, contra_string_no_stops, contra2_string_no_stops, moschaea_string_no_stops, vigonze_string_no_stops, vergiliana_string_no_stops, zanitonella_string_no_stops]
lemmas = [dererumnatura_lemmas, pharsalia_lemmas, thebaid_lemmas, africa_lemmas, christiad_lemmas, de_partu_virginis_lemmas, commedia_no_stops, morgante_no_stops, orlando_innamorato_no_stops, orlando_furioso_no_stops, italia_liberata_no_stops, gerusalemme_liberata_no_stops, macaronea_lemmas, magnificus_lemmas, baldus_lemmas, eglogae_lemmas, contra_lemmas, contra2_lemmas, moschaea_lemmas, vigonze_lemmas, vergiliana_lemmas, zanitonella_lemmas]

data = list(zip(title, date, token_num, fulltext, token_strings, lemmas))

df = pd.DataFrame(data, columns=["title", "date", "number of unique words", "full_text", "processed_tokens_string", "lemmas"])
df.to_csv("data.csv", index=False)

# Why Count Hapax Legonmena?

I counted hapax legomena because this number can be stastically relevant for studying literary artifacts. It has been a heuristic for:

1) <i>Authorship</i>. For example, W.P. Workman counted hapaxes in Shakespeares' works (Workman, H. P. “The Hapax Legomena of St. Paul.” ExpTim 7, 1896, 418–19).

2) <i>Language</i>. All the Latin and MP texts in this study have a higher number of hapax than the Italian texts.

2) <i>Corpus integrity</i> A high number can mean a corpus is too small or no longer exists.  

In [ ]:
# Assigns a list of the hapax to a variable

from nltk.probability import FreqDist

dererumnatura_unique = FreqDist(dererumnatura_no_stops)
dererumnatura_hapaxes = dererumnatura_unique.hapaxes()
aeneid_unique = FreqDist(aeneid_no_stops)
aeneid_hapaxes = aeneid_unique.hapaxes()
thebaid_unique = FreqDist(thebaid_no_stops)
thebaid_hapaxes = thebaid_unique.hapaxes()
pharsalia_unique = FreqDist(pharsalia_no_stops)
pharsalia_hapaxes = pharsalia_unique.hapaxes()
africa_unique = FreqDist(africa_no_stops)
africa_hapaxes = africa_unique.hapaxes()
commedia_unique = FreqDist(commedia_no_stops)
commedia_hapaxes = commedia_unique.hapaxes()
morgante_unique = FreqDist(morgante_no_stops)
morgante_hapaxes = morgante_unique.hapaxes()
orlando_innamorato_unique = FreqDist(orlando_innamorato_no_stops)
orlando_innamorato_hapaxes = orlando_innamorato_unique.hapaxes()
orlando_furioso_unique = FreqDist(orlando_furioso_no_stops)
orlando_furioso_hapaxes = orlando_furioso_unique.hapaxes()
italia_liberata_unique = FreqDist(italia_liberata_no_stops)
italia_liberata_hapaxes = italia_liberata_unique.hapaxes()
gerusalemme_liberata_unique = FreqDist(gerusalemme_liberata_no_stops)
gerusalemme_liberata_hapaxes = gerusalemme_liberata_unique.hapaxes()
faerie_queen_unique = FreqDist(faerie_queen_no_stops)
faerie_queen_hapaxes = faerie_queen_unique.hapaxes()
venus_and_adonis_unique = FreqDist(venus_and_adonis_no_stops)
venus_and_adonis_hapaxes = venus_and_adonis_unique.hapaxes()
paradise_lost_unique = FreqDist(paradise_lost_no_stops)
paradise_lost_hapaxes = paradise_lost_unique.hapaxes()
finnagans_wake_unique = FreqDist(finnagans_wake_no_stops)
finnagans_wake_hapaxes = finnagans_wake_unique.hapaxes()
ulysses_unique = FreqDist(ulysses_no_stops)
ulysses_hapaxes = ulysses_unique.hapaxes()
baldus_unique = FreqDist(baldus_no_stops)
baldus_hapaxes = baldus_unique.hapaxes()

In [ ]:
# Makes a dataframe with columns for title, language, number of hapax, and total number of words

t = ["De Rerum Natura", "Aeneid", "Pharsalia", "Thebaid", "Africa", "Commedia", "Morgante", "L'Innamoramento de Orlando", "Orlando furioso", "L'Italia liberata dai Goti", "Gerusalemme liberata", "Faerie Queene", "Venus and Adonis", "Paradise Lost", "Ulysses", "Finnagan's Wake", "Baldus",]
l = ["Latin", "Latin", "Latin", "Latin", "neo-Latin", "Italian", "Italian", "Italian", "Italian", "Italian", "Italian", "English", "English", "English", "English", "Macaronic", "Macaronic"]
h = [len(dererumnatura_hapaxes), len(aenied_hapaxes), len(pharsalia_hapaxes), len(thebaid_hapaxes), len(africa_hapaxes), len(commedia_hapaxes), len(morgante_hapaxes), len(orlando_innamorato_hapaxes), len(orlando_furioso_hapaxes), len(italia_liberata_hapaxes), len(gerusalemme_liberata_hapaxes), len(faerie_queen_hapaxes), len(venus_and_adonis_hapaxes), len(paradise_lost_hapaxes), len(ulysses_hapaxes), len(finnagans_wake_hapaxes), len(baldus_hapaxes)]
w = [len(dererumnatura_tokens_nopun), len(aeneid_tokens_nopun), len(pharsalia_tokens_nopun), len(thebaid_tokens_nopun), len(africa_tokens_nopun), len(commedia_tokens_nopun), len(morgante_tokens_nopun), len(orlando_innamorato_tokens_nopun), len(orlando_furioso_tokens_nopun), len(italia_liberata_tokens_nopun), len(gerusalemme_liberata_tokens_nopun), len(faerie_queen_nopun), len(venus_and_adonis_nopun), len(paradise_lost_nopun), len(ulysses_nopun), len(finnagans_wake_nopun), len(baldus_tokens_nopun)]
d = list(zip(t, l, h, w))
df = pd.DataFrame(d, columns=["Title", "Language", "Hapax", "Words"])

In [4]:
# Makes a scatterplot of unique words vs. total words with dots classified by language

sns.set()
g = sns.scatterplot(x='Hapax',
                y='Words',
                hue="Language", 
                style="Language",
                palette="Set1",
                data=df).set(title = 'Unique Words vs Total Words')
plt.legend(bbox_to_anchor=(1, 1), loc=2)
plt.savefig('Scatterplot.png', dpi=400, bbox_inches='tight')

NameError: name 'sns' is not defined

# Scatterplot 
<img src="../Scatterplot.png">

In [ ]:
# Makes a barplot showing the total number of hapax in the poems, side by side

sns.barplot(x='Hapax',
            y='Title',
            palette="Blues_d",
            data=df).set(title = 'Number of Unique Words');
plt.savefig('Barplot.png', dpi=400, bbox_inches='tight')

# Barchart
<img src="../Barplot.png">


# Zipf's law

Noticing the higher number of rare words in MP, I made rank-frequency distribution charts that visualize the degree to which the corpus adheres to an ideal Zipfian regression plot. George Zipf discovered that the relative frequency of words is inversely proportional to their rank:  

<img src="../zipf.png">

My charts are based on the famous regression chart included in George Zipf's classic 1949 book, <i>Human Behavior and The Principle of Least Effort</i>. Notice that Zipf tested his discovery on James Joyce's <i>Ulysses</i>, which has many unusual words:

<img src="../zip.png">

In [ ]:
# Makes a dataframe with _tokens variables for log log scatterplots

counter = Counter(aeneid_tokens_nopun)
df = pd \
    .DataFrame.from_dict(counter, orient='index') \
    .sort_values(0, ascending=False) \
    .reset_index() \
    .rename(columns={'index':'word', 0:'count'})
df.index = df.index + 1

In [ ]:
# Makes a column in the df with the rank frequency proportions

df['proportion'] = df['count']/df['count'].sum()

vocab_size, _ = df.shape
n_tokens = df['count'].sum()

print(f'Vocabulary size: {vocab_size:>8,}\nNumber of tokens: {n_tokens:,}')

In [ ]:
def classic_zipf(N, k, s=1):
    return (1/k**s)/(np.sum(1/(np.arange(1, N+1)**s)))

N = n_tokens

for i in range(0, 3):
    print(f'N={N:,}, k={10**i:<3}: {classic_zipf(N, 10**i):.4f}')

In [ ]:
%%time
vectorize_classic_zipf = np.vectorize(lambda x: classic_zipf(N, x))

df['predicted_proportion'] = vectorize_classic_zipf(df.index.values)

In [ ]:
import pandas as pd
import numpy as np

import plotly.offline as py
import plotly.figure_factory as ff
import plotly.graph_objs as go

import plotly
plotly.offline.init_notebook_mode(connected=True)

import spacy
import collections

import scipy

In [ ]:
x = np.log(df.index.values)
y = np.log(df['predicted_proportion'] * n_tokens)

trace = go.Scatter(
    x = x,
    y = y,
    mode = 'markers'
)

layout = go.Layout(
    xaxis = {
        'title': 'Rank'
    },
    yaxis = {
        'title': 'Frequency'
    }
)
data = [trace]
fig = go.Figure(data=data, layout=layout)

# Plot and embed in ipython notebook!
py.iplot(fig)

In [ ]:
x = np.log(df.index.values)
y = np.log(df['count'])

trace1 = go.Scatter(
    x = x,
    y = y,
    mode = 'markers',
    name = 'Actual',
)

x = np.log(df.index.values)
y = np.log(df['predicted_proportion'] * n_tokens)

trace2 = go.Scatter(
    x = x,
    y = y,
    mode = 'markers',
    name = 'Ideal'
)

data = [trace1, trace2]
fig = go.Figure(data=data, layout=layout)
fig.update_layout(
    title="Zanitonella",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
py.iplot(fig)

# Results
 
<img src="../charts.png">

<img src="../finbal.png">

In [ ]:
# Uses the week 5 code to loop through the files of macaronic poetry, turn the list of tokens into long string,
# and make it a dataframe

for_dataframe = {}

for file in dir_path:
    with open("/Users/matthewmason/cltk/corpus/" + file, "r", encoding="utf-8") as to_open:
         for_dataframe[file] = to_open.read()
            
mpcorpus = (pd.DataFrame.from_dict(for_dataframe, 
                                       orient = "index")
                .reset_index().rename(index = str, 
                                      columns = {"index": "File", 0: "Text"}))

mpcorpus = (pd.DataFrame.from_dict(for_dataframe, 
                                       orient = "index")
                .reset_index().rename(index = str, 
                                      columns = {"index": "File", 0: "Text"}))

mpcorpus["Text_processed"] = mpcorpus["Text"].map(lambda x: re.sub('[.,:;—?!"*„)(-]', '', x))

mpcorpus["Text_processed"] = mpcorpus["Text_processed"].map(lambda x: x.lower())

mpcorpus['Text_processed'] = mpcorpus['Text_processed'].str.replace('\d+', '')

list(mpcorpus[0:1]["Text_processed"])

long_string = ','.join(list(mpcorpus["Text_processed"].values))
long_string

word_tokenizer = WordTokenizer('latin')
mptokens = word_tokenizer.tokenize(long_string)
mp_nostops = [word for word in mptokens if not word in STOPS_LIST]

counter = Counter(mp_nostops)
df = pd \
    .DataFrame.from_dict(counter, orient='index') \
    .sort_values(0, ascending=False) \
    .reset_index() \
    .rename(columns={'index':'word', 0:'count'})
df.index = df.index + 1

df['proportion'] = df['count']/df['count'].sum()
df.head()

vocab_size, _ = df.shape
n_tokens = df['count'].sum()

print(f'Vocabulary size: {vocab_size:>8,}\nNumber of tokens: {n_tokens:,}')

def classic_zipf(N, k, s=1):
    return (1/k**s)/(np.sum(1/(np.arange(1, N+1)**s)))

N = n_tokens

for i in range(0, 3):
    print(f'N={N:,}, k={10**i:<3}: {classic_zipf(N, 10**i):.4f}')
    
%%time
vectorize_classic_zipf = np.vectorize(lambda x: classic_zipf(N, x))

df['predicted_proportion'] = vectorize_classic_zipf(df.index.values)

x = np.log(df.index.values)
y = np.log(df['predicted_proportion'] * n_tokens)

trace = go.Scatter(
    x = x,
    y = y,
    mode = 'markers'
)

layout = go.Layout(
    xaxis = {
        'title': 'Rank'
    },
    yaxis = {
        'title': 'Frequency'
    }
)
data = [trace]
fig = go.Figure(data=data, layout=layout)

py.iplot(fig)

In [ ]:
x = np.log(df.index.values)
y = np.log(df['count'])

trace1 = go.Scatter(
    x = x,
    y = y,
    mode = 'markers',
    name = 'Actual',
)

x = np.log(df.index.values)
y = np.log(df['predicted_proportion'] * n_tokens)

trace2 = go.Scatter(
    x = x,
    y = y,
    mode = 'markers',
    name = 'Ideal'
)

data = [trace1, trace2]
fig = go.Figure(data=data, layout=layout)
fig.update_layout(
    title="MP Corpus",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
py.iplot(fig)

<img src="../MP.png">

Future work